# **Mini Project Semester 6**<br>
#Topic: Generating HTML Code From UI Images<br>
Group Members:<br>
1.) Himanshu Bhawnani (IIB2020035)
<br> 2.) Shreya Shrotriya (IIT2020501)<br>
3.) Ishaan Kaustav (IIT2020261)<br>
4.) Shreyans Jain (IIT2020238)<br>
5.) Mantraraj Gotecha (IIT2020269)


In [1]:
#importing necessary modules
from util import UIDataset, Vocabulary
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
from torch.utils.data import DataLoader
from model import *
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

# **Importing Dataset**

The dataset is available in the drive link: https://drive.google.com/drive/folders/1h53d1FDpqYsAoeX26Fcd2xC7xNWv5iWh?usp=sharing

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
dataset = UIDataset('/content/gdrive/MyDrive/dataset/training', 'voc.pkl')

# Training

In [3]:
net = Pix2Code().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0001)

In [3]:
for epoch in range(10):
    net.zero_grad()
    for j, data in enumerate(dataset):
        image, context, prediction = data
        image = image.unsqueeze(0).cuda()
        context = context.unsqueeze(0).cuda()
        prediction = prediction.cuda()
        output = net(image, context)
        output = output.squeeze(0)
        prediction = torch.argmax(prediction, 1)
        loss = criterion(output, prediction)
        loss.backward()
        if j%10 == 0:
            optimizer.step()
            print('Loss: {}, Epoch: {}'.format(loss.data, epoch))
            net.zero_grad()

torch.save(net.state_dict(), './pix2code.weights')

# Testing

In [3]:
net = Pix2Code()
net.load_state_dict(torch.load('./pix2code.weights'))
net.cuda().eval()

In [3]:
test_data = UIDataset('./dataset/evaluation', 'voc.pkl')
vocab = Vocabulary('voc.pkl')

In [3]:
image, *_ = test_data.__getitem__(np.random.randint(len(test_data)))
t = transforms.ToPILImage()
image = image.unsqueeze(0)
t(image.squeeze())

In [3]:
image = image.cuda()
image = image.repeat(1, 3, 1, 1)  # Repeat the image along the channel dimension

ct = []
ct.append(vocab.to_vec(' '))
ct.append(vocab.to_vec('<START>'))
output = ''
for i in range(200):
    context = torch.tensor(ct).unsqueeze(0).float().cuda()
    index = torch.argmax(net(image, context), 2).squeeze()[-1:].squeeze()
    v = vocab.to_vocab(int(index))
    if v == '<END>':
        break
    output += v
    ct.append(vocab.to_vec(v))

with open('output.gui', 'w') as f:
    f.write(output)

print(output)

This is a GUI code, in order to get the ouput from it, a GUI compiler need to be developed.

# **---------------------------------------Thanks-----------------------------------------------**